# <center> Линейная алгебра в контексте линейных методов. Практика.

## <center> Прогнозирование выработки газа на скважинах.

## Постановка задачи

У Василия, основателя компании «Газ-Таз-Ваз-Нефть», дела идут в гору: у него уже функционирует 200 скважин для добычи газа. В этом году он открывает 30 новых скважин. Однако в целях оптимизации расходов и повышения дохода Василию необходимо оценить, сколько денег будет приносить ему каждая из скважин, а также понять, какие факторы (параметры скважин) потенциально сильнее всего повлияют на объём добычи газа. Для этого Василий решил нанять вас как специалиста в области Data Science.

Василий представляет вам набор данных о добыче газа на своих скважинах. Файл с данными вы можете скачать на платформе.

**Признаки в данных:**

* Well — идентификатор скважины;
* Por — пористость скважины (%);
* Perm — проницаемость скважины;
* AI — акустический импеданс ($кг/м^2 * 10^6$);
* Brittle — коэффициент хрупкости скважины (%);
* TOC — общий органический углерод (%);
* VR — коэффициент отражения витринита (%);
* Prod — добыча газа в сутки (млн. кубических футов).

**Ваша задача** — построить регрессионную модель, которая прогнозирует выработку газа на скважине (целевой признак — Prod) на основе остальных характеристик скважины, и проинтерпретировать результаты вашей модели.

Разделим задачу на две части:
* В первой мы построим простейшую модель линейной регрессии, проанализируем результаты её работы и выберем наиболее значимые факторы для прогнозирования.
* Во второй мы займёмся построением модели полиномиальной регрессии с регуляризацией и посмотрим на итоговые результаты моделирования.


Импортируем необходимые библиотеки:

In [45]:
# Импорт библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
import seaborn as sns # библиотека для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графико

import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import metrics


%matplotlib inline

Прочитаем исходные данные:

In [46]:
data = pd.read_csv('data/unconv.csv')
data.head()

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
0,1,12.08,2.92,2.80,81.40,1.16,2.31,4165.196191
1,2,12.38,3.53,3.22,46.17,0.89,1.88,3561.146205
2,3,14.02,2.59,4.01,72.80,0.89,2.72,4284.348574
3,4,17.67,6.75,2.63,39.81,1.08,1.88,5098.680869
4,5,17.52,4.57,3.18,10.94,1.51,1.90,3406.132832


## Практика: линейная регрессия по методу наименьших квадратов

Для начала построим простейшую модель линейной регрессии, проанализируем результаты её работы и выберем наиболее значимые факторы для прогнозирования.

В первой части вам предстоит выполнить задания 5.0–5.6. Максимальное количество баллов, которое можно получить, — 9.

### Задание 5.0. (не оценивается)

Проведите небольшой разведывательный анализ, чтобы поближе познакомиться с исходными данными. Постройте несколько визуализаций, которые помогут вам понять, как устроены исходные признаки, а также смогут продемонстрировать взаимосвязь факторов. 

Сделайте промежуточные выводы из проведённого разведывательного анализа.

In [47]:
# Ваш код здесь
display(data.info())
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Well     200 non-null    int64  
 1   Por      200 non-null    float64
 2   Perm     200 non-null    float64
 3   AI       200 non-null    float64
 4   Brittle  200 non-null    float64
 5   TOC      200 non-null    float64
 6   VR       200 non-null    float64
 7   Prod     200 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 12.6 KB


None

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,100.500000,14.991150,4.330750,2.968850,48.161950,0.990450,1.964300,4311.219852
std,57.879185,2.971176,1.731014,0.566885,14.129455,0.481588,0.300827,992.038414
min,1.000000,6.550000,1.130000,1.280000,10.940000,-0.190000,0.930000,2107.139414
25%,50.750000,12.912500,3.122500,2.547500,37.755000,0.617500,1.770000,3618.064513
50%,100.500000,15.070000,4.035000,2.955000,49.510000,1.030000,1.960000,4284.687348
75%,150.250000,17.402500,5.287500,3.345000,58.262500,1.350000,2.142500,5086.089761
max,200.000000,23.550000,9.870000,4.630000,84.330000,2.180000,2.870000,6662.622385


In [48]:
categorical_features = ["Well", "Por", "Perm", "AI", "Brittle", "TOC", "VR"]

for feature in categorical_features:
    fig = px.histogram(data, x=feature,nbins=30,
    width=700,
    height=250)
    fig.show()

### Задание 5.1. (2 балла)

Постройте корреляционную матрицу факторов, включив в неё целевой признак. 

Для наглядности визуализируйте полученную матрицу с помощью тепловой карты. 

**Примечание.** *Для визуализации вы можете использовать любую из знакомых вам библиотек.*

На основе построенной корреляционной матрицы ответьте на следующий вопрос:

* Какие факторы сильнее всего коррелируют с целевой переменной?

Вычислите ранг и определитель полученной матрицы корреляций и приведите развёрнутые ответы на следующие вопросы:
* Является ли корреляционная матрица плохо обусловенной (близок ли её определитель к нулю)?
* Что вы можете сказать о наличии коллинераности/мультиколлинеарности в данных? 
* Применима ли в данном случае модель классической линейной регрессии по методу наименьших квадратов и почему? 

**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**2 балла**): 
    * приведён код для расчёта корреляционной матрицы;
    * приведён код для визуализации корреляционной матрицы в виде тепловой карты;
    * рассчитаны ранг и определитель корреляционной матрицы;
    * предоставлены обоснованные ответы на все поставленные вопросы. 

- Задание выполнено верно, но не учтено одно условие (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [49]:
display(data.corr())
display(f"Ранг матрицы: {np.linalg.matrix_rank(data.corr())}")
display(f"Опредилитель матрицы: {np.linalg.det(data.corr())}")

fig = px.imshow(data.corr(),text_auto=True,width=1000,height=1000)
fig.show()


,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.000000,0.068927,0.077928,0.041483,-0.079252,0.022624,-0.007279,0.026817
Por,0.068927,1.000000,0.760546,-0.461549,-0.218570,0.711831,0.111860,0.861910
Perm,0.077928,0.760546,1.000000,-0.239636,-0.124017,0.471746,0.051023,0.727426
AI,0.041483,-0.461549,-0.239636,1.000000,0.127599,-0.531864,0.499143,-0.390835
Brittle,-0.079252,-0.218570,-0.124017,0.127599,1.000000,-0.214282,0.317929,0.237155
TOC,0.022624,0.711831,0.471746,-0.531864,-0.214282,1.000000,0.299483,0.654445
VR,-0.007279,0.111860,0.051023,0.499143,0.317929,0.299483,1.000000,0.323182
Prod,0.026817,0.861910,0.727426,-0.390835,0.237155,0.654445,0.323182,1.000000


'Ранг матрицы: 8'

'Опредилитель матрицы: 0.0007299388072652069'

> - Факторы:[`Por`, `TOC`, `Perm`] сильнее всего коррелируют с целевой переменной **`Prod`**;
> - Данная корреляционная матрица является плохо обусловенной так как ее определитель близок к 0 ~ 0.0007;
> - Имеется 2 пары взаимо корелируемых признака [`Por`, `TOC`],[`Por`, `Perm`] с кореляцией >0.7 что говорит о наличии Мультиколлинеарности;
> - В данном случае модель классической линейной регрессии по методу наименьших квадратов **применима** так как Мультиколлинеарность не мешат построению модели но будут проблемы с интерпритацией вектара коэфицентов так как они могут поменять знаки на противоположные или улететь в космос и не будут объесняться бизнес логикой.

### Задание 5.2. (2 балла)

Создайте матрицу наблюдений `X` и вектор правильных ответов `y`. В качестве факторов для матрицы наблюдений возьмите все имеющиеся в данных признаки. 

Постройте модель линейной регрессии по методу наименьших квадратов. Для этого воспользуйтесь матричной формулой МНК и инструментарием библиотеки numpy. 

Выведите на экран полученные оценки коэффициентов модели, округлённые до целого.

Приведите подробные выводы по полученным результатам, ответив на следующие вопросы:
* Есть ли в ваших данных признаки, которые, согласно модели, можно назвать неинформативными, то есть они не оказывают влияния на целевую переменную или оказывают шумовое влияние?
* Приведите интерпретацию нескольких (двух или трёх) коэффициентов полученной модели линейной регрессии.


**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**2 балла**): 
    * приведён код для нахождения параметров модели линейной регрессии с помощью матричной формулы МНК;
    * на основе полученных параметров даны верные ответы на поставленные вопросы;
    * приведена корректная интерпретация нескольких коэффициентов полученной модели линейной регрессии.

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [50]:
X = data[["Well", "Por", "Perm", "AI", "Brittle", "TOC", "VR"]].values
y = np.array(data["Prod"])

w = np.linalg.inv(X.T@X)@X.T@y
w.round()

array([  -0.,  185.,  158., -676.,   20., -273., 1125.])

> - Признак `Well` не информативен так как это индетификатор скважины, что и подтвердили как кореляция так и коэфицент.
> - положительно сказываются `VR коэффициент отражения витринита (%)` на 1% приходится 1125 млн. куб. футов,
  в свою очередь `TOC — общий органический углерод (%)` за 1% предпологаемый уровень добычи снижается на 273 млн. куб. футов.

### Задание 5.3. (1 балл)

Теперь потренируемся строить предсказание для наблюдений целевой переменной. 

**а)** Постройте прогноз выработки газа для скважины со следующими параметрами:

```python
{
    'Well': 106.0,
    'Por': 15.32,
    'Perm': 3.71,
    'AI': 3.29,
    'Brittle': 55.99,
    'TOC': 1.35,
    'VR': 2.42
 }
```

Рассчитайте абсолютную ошибку построенного вами прогноза для предложенной скважины (в миллионах кубических футов в день), если известно, что на этой скважине производится `4748.315024` миллионов кубических футов газа в день. 

**б)** Постройте прогноз выработки газа для всех скважин из обучающего набора данных. Выберите метрику, по которой вы будете оценивать качество регрессионных моделей и сравнивать их между собой (можно использовать несколько метрик, но не более двух).

Рассчитайте значение выбранной метрики для вашей модели. Приведите интерпретацию полученного результата. 

**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**1 балл**): 
    * приведён код для построения прогноза выработки газа на указанной скважине и рассчитана абсолютная ошибка прогноза для этой скважины;
    * выбрана корректная для поставленной задачи метрика (или метрики) оценки качества модели и приведён код для расчёта этой метрики на всём обучающем наборе данных;
    * приведена корректная интерпретация полученного результата.

- Задание выполнено верно, но не учтено одно из условий, или задание выполнено неверно (**0 баллов**).

In [51]:
parm = {
    'Well': 106.0,
    'Por': 15.32,
    'Perm': 3.71,
    'AI': 3.29,
    'Brittle': 55.99,
    'TOC': 1.35,
    'VR': 2.42
 }
b = np.array(list(parm.values()))

vec_pred = 0
for i in range(X.shape[1]):
    vec_pred += b[i]*w[i]
MAE = 4748.315024 - vec_pred
print(f'Предпологаеиый объем добычи газа {vec_pred} и ошибка для этого расчета {MAE}')

y_pred = 0
for i in range(X.shape[1]):
    y_pred = y_pred + X[:,i]*w[i]

print(f'MAPE для всей выборки: {round(metrics.mean_absolute_percentage_error(y, y_pred)* 100,2)} %')
print(f'R^2 для всей выборки: {round(metrics.r2_score(y, y_pred),2)}')

Предпологаеиый объем добычи газа 4631.708570600747 и ошибка для этого расчета 116.60645339925304
MAPE для всей выборки: 3.98 %
R^2 для всей выборки: 0.95


> - Было сделано предсказание по предоставленым данным ошибка равна 116.6 единиц ~ 2.5% очень хороший результа, но что же покажет модель на всех результатах?
> - После предикта получили вектр результатов , вычеслины следующие метрики MAPE и R2 среднея обсолютная процентная ошибка  = 3.8% очень хороший показатель значит модель ошибается в среднем на 3.8% . А R2 показывает насколько наша модель подстроенна под данные.

### Задание 5.4. (1 балл)

Настало время анализа построенной модели. Посмотрите на коэффициенты и сравните их знаки со значениями выборочных корреляций между целевым признаком и факторами, которые вы нашли ранее.

Дайте развёрнутые ответы на следующие вопросы:

* Есть ли в вашей модели фактор, при котором коэффициент в модели линейной регрессии противоречит соответствующему коэффициенту корреляции? Например, корреляция говорит, что зависимость между фактором и целевой переменной прямая, а модель говорит обратное. Если таких факторов несколько, укажите их все.
* Как вы считаете, с чем связано полученное противоречие результатов?

**Критерии оценивания:**

- Задание выполнено верно, даны корректные развёрнутые ответы на все поставленные в задании вопросы (**1 балл**). 

- Задание выполнено неверно, даны некорректные ответы на один или несколько поставленных в задании вопросов (**0 баллов**).

In [52]:
# обьединим вектор весов и вектор кореляций целевого признака с другими признаками
a = pd.concat([pd.DataFrame(w),pd.DataFrame(data.corr()["Prod"][:-1].values)],axis=1)
a.index = ['Well', 'Por', 'Perm', 'AI', 'Brittle', 'TOC', 'VR']
a.columns = ["w_hat","corr"]
a

,w_hat,corr
Well,-0.043450,0.026817
Por,184.616889,0.861910
Perm,157.557782,0.727426
AI,-675.731071,-0.390835
Brittle,19.527421,0.237155
TOC,-272.983826,0.654445
VR,1124.705458,0.323182


> Сразу в глаза бросается это изменения знаков у `Well`, `TOC` что и было примерно ясно еще по кориляции так как `TOC` имел высокую кореляцию с `POR` а `Well` имел очень маленькие кореляции со всеми даже с целевой и просто мешал. Вывод присутсвует вырожденость матрицы. Но в тоже время признаки [`Por`, `Perm`] не поменяли знаки и не ясно как они влияют на модель , так что наличие высокой кореляции это еще 1 повод для удаления признака но не точный показатель что показатель будет мешать.

### Задание 5.5. (2 балла)

* Исключите из данных сильно коррелированные между собой факторы. Под сильной корреляцией в данной задаче будем понимать значения выше `0.7`.

Выбирая, какой из коррелированных факторов оставить, руководствуйтесь коэффициентом корреляции с целевой переменной: оставляйте тот фактор, который больше всего коррелирует с целевой переменной (объёмом добычи газа).

* Также исключите из данных факторы, для которых корреляция с целевой переменной меньше `0.05`.

Постройте модель линейной регрессии на обновлённых после удаления факторов данных по методу наименьших квадратов. Для этого используйте матричную формулу и библиотеку numpy. Выведите значение полученных коэффициентов, округлённых до целого.

Сделайте прогноз для всего обучающего набора данных и рассчитайте выбранную вами ранее метрику (или метрики). 

Дайте подробные ответы на следующие вопросы:

* Приведите интерпретацию нескольких полученных коэффициентов (двух или трёх). Сравните их значения с полученными ранее.
* Приведите интерпретацию полученных значений метрик. Сравните их значения с полученными ранее. 
* Удалось ли вам побороть противоречие коэффициентов линейной регрессии и коэффициентов корреляции?

**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**2 балла**): 
    * отобраны верные факторы;
    * на основе отобранных факторов построена модель линейной регрессии с помощью матричной формулы МНК и найдены параметры модели, соответствующие каждому из факторов;
    * построен прогноз для всего обучающего набора данных и рассчитано значение метрики (метрика должна быть выбрана корректно);
    * даны ответы на все поставленные в задании вопросы.

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено верно, не учтено несколько из условий (**0 баллов**).

In [53]:
X = data.drop(["Prod","Well","Perm"],axis=1).values
y = data["Prod"]
w=np.linalg.inv(X.T@X)@X.T@y
print(w.reshape(5,1))

y_pred = 0
for i in range(X.shape[1]):
    y_pred = y_pred + X[:,i]*w[i]
print(f'MAPE для всей выборки: {round(metrics.mean_absolute_percentage_error(y, y_pred)* 100,2)} %')
print(f'R^2 для всей выборки: {round(metrics.r2_score(y, y_pred),2)}')

[[ 243.76441648]
 [-584.39524662]
 [  20.65020245]
 [-170.9830026 ]
 [ 804.9278732 ]]
MAPE для всей выборки: 4.99 %
R^2 для всей выборки: 0.92


> - При увелечении на 1 единицу коэффициент отражения витринита (%) добыча газа в сутки увеличится на 805 единиц, так же при увелечении на 1 единицу коэффициент общего органического углерода (%) то добыча газа в сутки упадёт на 171 единицы по сравнению с предыдущими, показателями изменились, но не колосально
> - Метрики чуть ухудшились по сравнению с предыдущими значениями метрик
> - Противоречия сохранились

### Задание 5.6. (1 балл)

Наконец, давайте построим модель линейной регрессии из библиотеки `scikit-learn (sklearn)` и сравним результаты её работы с теми, что нам удалось получить вручную. 

Постройте модель линейной регрессии на обновлённых после удаления факторов данных по методу наименьших квадратов. Для этого воспользуйтесь классом `LinearRegression` из библиотеки `sklearn`. Выведите значения полученных коэффициентов, округлённые до целого.

Сделайте прогноз для всего обучающего набора данных и рассчитайте значение выбранной вами метрики (или метрик).

Сравните результаты библиотечной реализации с тем, что вы получили ранее. 

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * на основе отобранных факторов построена модель линейной регрессии из библиотеки sklearn и найдены параметры модели, соответствующие каждому из факторов;
    * построен прогноз для всего обучающего набора данных и рассчитано значение метрики (метрика должна быть выбрана корректно);
    * приведён вывод о соответствии результатов, полученных вручную и с помощью библиотеки.
- Задание выполнено неверно, не учтено одно или несколько условий (**0 баллов**).

In [54]:
from sklearn.linear_model import LinearRegression
lr_full = LinearRegression(fit_intercept=False)
#Обучаем модель — ищем параметры по МНК
lr_full.fit(X, y)



y_pred_lr = lr_full.predict(X)
print(f'MAPE для всей выборки: {round(metrics.mean_absolute_percentage_error(y, y_pred_lr),2)} %')
print(f'R^2 для всей выборки: {round(metrics.r2_score(y, y_pred_lr),2)}')

MAPE для всей выборки: 0.05 %
R^2 для всей выборки: 0.92


> результаты индетичны, но с интерсептом результаты лутче

## Практика: полиномиальная регрессия и регуляризация

Мы продолжаем работать над задачей от владельца компании «Газ-Таз-Ваз-Нефть» Василия.

Ранее мы построили модель линейной регрессии, которая прогнозирует выработку газа на скважине. Для этого мы с помощью матрицы корреляций и рассуждений отобрали некоррелированные, значимые для предсказания признаки. **Далее мы будем использовать именно их (см. задание 5.5).**

Мы хотим улучшить результат — уменьшить ошибку прогноза. Для этого мы воспользуемся моделью полиномиальной регрессии третьей степени. Однако теперь мы знаем, что полиномиальным моделям очень легко переобучиться под исходную выборку. Так как данных у нас не так много (всего 200 скважин), то для контроля качества модели мы будем использовать кросс-валидацию. 

Приступим! Выполните задания 8.1–8.5:


In [55]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

### Задание 8.1. (1 балл)

Стандаризируйте признаки с помощью `StandartScaler` из библиотеки `sklearn`. 

Затем сгенерируйте полиномиальные признаки третьего порядка на факторах, которые вы выбрали для обучения моделей. Для этого воспользуйтесь генератором полиномов `PolynomialFeatures` из библиотеки `sklearn`. Параметр `include_bias` установите в значение `False`.

Выведите на экран, сколько факторов у вас получилось после генерации полиномиальных признаков.

**Важно:** стандартизацию необходимо произвести до генерации полиномиальных факторов!

Обучите модель линейной регрессии из библиотеки `sklearn` (`LinearRegression`) на полученных полиномиальных факторах.

Используя кросс-валидацию оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * на основе отобранных факторов сгенерированы полиномиальные признаки третьего порядка;
    * построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).
- Задание выполнено неверно, не учтено одно или несколько из условий (**0 баллов**).

In [56]:
X = data.drop(["Prod","Well","Perm","TOC"],axis=1).values
y = data["Prod"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

poly = PolynomialFeatures(degree=3,include_bias=False)
X = poly.fit_transform(X)
print(f"После генерации полиномиальных признаков, факторов получилось: {X.shape[1]}")
print()
lr = LinearRegression()

cv_result = cross_validate(lr, X, y, scoring=["neg_mean_absolute_percentage_error","r2"], cv=5 , return_train_score=True)

print('MAPE на тренировочных фолдах: {:.2f} %'.format(abs(cv_result['train_neg_mean_absolute_percentage_error'].mean()*100)))
print('R2 на тренировочных фолдах: {:.2f}'.format(abs(cv_result['train_r2'].mean())))
print()
print('MAPE на валидационных фолдах: {:.2f} %'.format(abs(cv_result['test_neg_mean_absolute_percentage_error'].mean()*100))) 
print('R2 на валидационных фолдах: {:.2f}'.format(abs(cv_result['test_r2'].mean()))) 

После генерации полиномиальных признаков, факторов получилось: 34

MAPE на тренировочных фолдах: 1.77 %
R2 на тренировочных фолдах: 0.99

MAPE на валидационных фолдах: 2.68 %
R2 на валидационных фолдах: 0.97


> Не значительное увелечение MAPE на тестовых данных, если и есть пееробучение то не значительное

### Задание 8.2. (2 балла)

Теперь попробуем воспользоваться линейной регрессией с регуляризацией. Для начала возьмём $L_1$-регуляризацию.

Обучите модель `Lasso` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициент регуляризации (`alpha`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию, оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (Lasso), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [57]:
from sklearn.model_selection import RandomizedSearchCV
X = data.drop(["Prod","Well","Perm","TOC"],axis=1).values
y = data["Prod"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

poly = PolynomialFeatures(degree=3,include_bias=False)
X = poly.fit_transform(X)

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_distributions = {'alpha': list(np.linspace(1, 10, 40, dtype=float))}
            
random_search = RandomizedSearchCV(
    estimator=Lasso(random_state=42), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  

random_search.fit(X,y)

lasso = Lasso(alpha=random_search.best_params_["alpha"],max_iter=10000)
cv_result_l = cross_validate(lasso, X, y, scoring=["neg_mean_absolute_percentage_error","r2"], cv=5 , return_train_score=True)

print('MAPE на тренировочных фолдах: {:.2f} %'.format(abs(cv_result_l['train_neg_mean_absolute_percentage_error'].mean()*100)))
print('R2 на тренировочных фолдах: {:.2f}'.format(abs(cv_result_l['train_r2'].mean())))
print()
print('MAPE на валидационных фолдах: {:.2f} %'.format(abs(cv_result_l['test_neg_mean_absolute_percentage_error'].mean()*100))) 
print('R2 на валидационных фолдах: {:.2f}'.format(abs(cv_result_l['test_r2'].mean()))) 
display(random_search.best_params_)

MAPE на тренировочных фолдах: 1.86 %
R2 на тренировочных фолдах: 0.99

MAPE на валидационных фолдах: 2.28 %
R2 на валидационных фолдах: 0.98


{'alpha': 6.307692307692308}

> метрики для тренировочной выборки ухудшились а вот для валидационной улучшились например MAPE уменьшилось на ~ 0.4%

### Задание 8.3. (2 балла)

Проделаем то же самое с $L_2$-регуляризацией.

Обучите модель `Ridge` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициент регуляризации (`alpha`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

In [58]:
X = data.drop(["Prod","Well","Perm","TOC"],axis=1).values
y = data["Prod"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

poly = PolynomialFeatures(degree=3,include_bias=False)
X = poly.fit_transform(X)

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_distributions = {'alpha': list(np.linspace(0.1, 1, 20, dtype=float))}
            
random_search = RandomizedSearchCV(
    estimator=Ridge(random_state=42,solver='svd'), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 20, 
    n_jobs = -1
)  

random_search.fit(X,y)

ridg = Ridge(alpha=random_search.best_params_["alpha"],solver='svd')
cv_result_r = cross_validate(ridg, X, y, scoring=["neg_mean_absolute_percentage_error","r2"], cv=5 , return_train_score=True)

print('MAPE на тренировочных фолдах: {:.2f} %'.format(abs(cv_result_r['train_neg_mean_absolute_percentage_error'].mean()*100)))
print('R2 на тренировочных фолдах: {:.2f}'.format(abs(cv_result_r['train_r2'].mean())))
print()
print('MAPE на валидационных фолдах: {:.2f} %'.format(abs(cv_result_r['test_neg_mean_absolute_percentage_error'].mean()*100))) 
print('R2 на валидационных фолдах: {:.2f}'.format(abs(cv_result_r['test_r2'].mean()))) 
display(random_search.best_params_)

MAPE на тренировочных фолдах: 1.81 %
R2 на тренировочных фолдах: 0.99

MAPE на валидационных фолдах: 2.70 %
R2 на валидационных фолдах: 0.98


{'alpha': 0.8105263157894737}

> L2 регулизация не справилась с поставленой задачей так как MAPE на валидационной хуже чем у обычной линейной регресии,

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (Ridge), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

### Задание 8.4. (2 балла)

Наконец, настало время комбинировать $L_1$ и $L_2$ -регуляризации.

Обучите модель `ElasticNet` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициенты регуляризации (`alpha` и `l1-ratio`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию, оцените среднее значение метрики MAPE на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (ElasticNet), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [59]:
X = data.drop(["Prod","Well","Perm","TOC"],axis=1).values
y = data["Prod"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

poly = PolynomialFeatures(degree=3,include_bias=False)
X = poly.fit_transform(X)

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_distributions = {'alpha': list(np.linspace(0.1, 1, 100, dtype=float)),
                       "l1_ratio":list(np.linspace(0.1, 1, 100, dtype=float))}
            
random_search = RandomizedSearchCV(
    estimator=ElasticNet(fit_intercept=False), 
    param_distributions=param_distributions, 
    scoring='neg_mean_absolute_percentage_error',
    cv=5, 
    n_iter = 5000, 
    n_jobs = -1
)  

random_search.fit(X,y)

elast = ElasticNet(alpha=random_search.best_params_['alpha'],l1_ratio= random_search.best_params_['l1_ratio'], max_iter=10000)
cv_result_e = cross_validate(elast, X, y, scoring=["neg_mean_absolute_percentage_error","r2"], cv=5 , return_train_score=True)

print('MAPE на тренировочных фолдах: {:.2f} %'.format(abs(cv_result_e['train_neg_mean_absolute_percentage_error'].mean()*100)))
print('R2 на тренировочных фолдах: {:.2f}'.format(abs(cv_result_e['train_r2'].mean())))
print()
print('MAPE на валидационных фолдах: {:.2f} %'.format(abs(cv_result_e['test_neg_mean_absolute_percentage_error'].mean()*100))) 
print('R2 на валидационных фолдах: {:.2f}'.format(abs(cv_result_e['test_r2'].mean()))) 
display(random_search.best_params_)

MAPE на тренировочных фолдах: 5.79 %
R2 на тренировочных фолдах: 0.91

MAPE на валидационных фолдах: 6.92 %
R2 на валидационных фолдах: 0.87


{'l1_ratio': 0.1, 'alpha': 0.44545454545454544}

> MAPE как на тренировчной так и на валидационной выборке ухудшился а время затрачено в 10ки раз больше, возможно не правельно подобраны параметры но в общем разница видна простые L1 и L2 регулязация спаравились лутчше

### Задание 8.5. (1 балл)

Завершением вашей работы будет сводная таблица результатов, которую вы представите на презентации заказчику Василию.

Составьте таблицу (DataFrame) со следующими столбцами (имена столбцов выберите самостоятельно):
* Наименование модели.
* Гиперпараметры (коэффициенты регуляризации, если таковые имеются), если нет — оставьте ячейку пустой.
* Использовались ли полиномиальные признаки при построении модели (Да/Нет или True/False).
* Значение выбранной метрики на тренировочных фолдах при кросс-валидации модели.
* Значение выбранной метрики на валидационных фолдах при кросс-валидации модели.

Сделайте финальный вывод по проделанной работе. Какая линейная модель из тех, что мы рассмотрели, показала наилучший результат с точки зрения качества на валидационных фолдах, а также с точки зрения переобученности?

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * верно составлена сводная таблица итоговых результатов построенных ранее моделей с указанными в задании столбцами;
    * приведены итоговые выводы по проделанной работе, указано, какая из рассмотренных моделей показала наилучший результат.

- Задание выполнено неверно, не учтено одно или несколько условий (**0 баллов**).

In [79]:
d = pd.DataFrame({
    "name_model": [lr,Lasso(),Ridge(),ElasticNet()],
    "Hyperparameters": ["",
                        f"alpha ~ {round(lasso.get_params()['alpha'],3)}",
                        f"alpha ~ {round(ridg.get_params()['alpha'],3)}",
                        f"alpha ~ {round(elast.get_params()['alpha'],3)} , l1_ratio = {round(elast.get_params()['l1_ratio'],3)}"],
    "Use_polynome": ["True","True","True","True"],
    "MAPE_train": [round(abs(cv_result['train_neg_mean_absolute_percentage_error'].mean()*100),2),
                   round(abs(cv_result_l['train_neg_mean_absolute_percentage_error'].mean()*100),2),
                   round(abs(cv_result_r['train_neg_mean_absolute_percentage_error'].mean()*100),2),
                   round(abs(cv_result_e['train_neg_mean_absolute_percentage_error'].mean()*100),2)],
    "MAPE_valid": [round(abs(cv_result['test_neg_mean_absolute_percentage_error'].mean()*100),2),
                   round(abs(cv_result_l['test_neg_mean_absolute_percentage_error'].mean()*100),2),
                   round(abs(cv_result_r['test_neg_mean_absolute_percentage_error'].mean()*100),2),
                   round(abs(cv_result_e['test_neg_mean_absolute_percentage_error'].mean()*100),2)]
})
d

,name_model,Hyperparameters,Use_polynome,MAPE_train,MAPE_valid
0,LinearRegression(),,True,1.77,2.68
1,Lasso(),alpha ~ 6.308,True,1.86,2.28
2,Ridge(),alpha ~ 0.811,True,1.81,2.70
3,ElasticNet(),"alpha ~ 0.445 , l1_ratio = 0.1",True,5.79,6.92


> Самые летчшие результы показала иодель Lasso так как средняя абсолютная ошибка в процентах состовляет 2.28% ~ на 4 десятых лутчше чем L2 регулизация.